# **Python Azure**
---

In [10]:
# 필요한 패키지 설치
!pip install azure-storage-blob azure-identity

     -------------------------------------- 135.5/135.5 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 90.0/90.0 kB 2.6 MB/s eta 0:00:00


In [11]:
import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [12]:
# Azure에서 복사한 '연결 문자열'을 이용하여 Blob 서버에 접속하기
connect_str = 'DefaultEndpointsProtocol=https;AccountName=labuser111storageaccount;AccountKey=*************************i5kTFlc4sHX0QBGFxm8Q3NBDk4/Xt7UJ+AStpdhafw==;EndpointSuffix=core.windows.net'             # 연결 문자열 ✅
blob_service_client = BlobServiceClient.from_connection_string(connect_str)     # 연결 문자열을 이용해서 Blob Service에 연결

In [13]:
container_name = 'test2container'

blob_service_client.create_container(container_name)

In [14]:
# Create a local directory to hold blob data
local_path = './data'
os.mkdir(local_path)

# Create a file in the local directory
local_file_name = 'welcome' + '.txt'
upload_file_path = os.path.join(local_path, local_file_name)

# Write text to the file
file = open(upload_file_path, mode='w')
file.write('Welcome to Python')
file.close()     # Close를 안하면 파일이 나중에 잘 깨진다. (0B 파일들)

In [17]:
blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file_name)
with open(file=upload_file_path, mode='rb') as data:
    blob_client.upload_blob(data)

In [ ]:
# blob 파일의 리스트를 확인한다. 
print('\nListing blobs')

blob_list = container_client.list_blobs()
for blob in blob_list:
  print('\t' + blob.name)

In [ ]:
# Download the blob to a local file
# Add 'DOWNLOAD' befere the .txt extendsion so you can see both files in the data directory 

download_file_path = os.path.join(local_path, 
                                  str.replace(local_file_name,'.txt','_DOWNLOAD.txt'))
print(download_file_path)
container_client = blob_service_client.get_container_client(container=container_name)

with open(file=download_file_path, mode='wb') as download_file:
  download_file.write(container_client.download_blob(blob.name).readall())

In [ ]:
# Clean Up
print('\nPress the Enter Key to begin clean up')
if input() == 'yes':

  print('Deleting blob container...')
  container_client.delete_container()

  print('Deleteing the local source and downloaded files....')
  os.remove(upload_file_path)
  os.remove(download_file_path)
  os.rmdir(local_path)

  print('Done')